# 03_Image_Augmentation_Trucks 

To increase the number of training images, we can take the originals and 
- convert to black and white
- rotate (up to 5 degrees)
- flip horizontally
- crop slightly

This notebook was designed to extend the number of images in the original training set, but it can also be run on any user selected directory.

## Imports

In [1]:
from PIL import Image, ImageEnhance
import os
import numpy as np

## Global configuration settings

In [2]:
# Set this to convert images in the lower directories
base_dir = '../images/trucks'

## Transformation functions

In [3]:
# For reproducibility during development and testing, do not change this value.
np.random.seed(2020)

In [4]:
# Remove .DS_store from the list of files
def rm_DS(mlist): 
    if ('.DS_Store' in mlist): mlist.remove('.DS_Store')
    return mlist

In [5]:

def make_hflip_version(path,fname):
                
    new_fname = 'hflip_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + '/' + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [6]:
def make_bw_version(path,fname):
 
    new_fname = 'bw_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + '/' + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.convert(mode='L')
    converted .save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [7]:
def make_rot_version(path,fname,rotation):
            
    new_fname = 'rot-'+ str(rotation) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + '/' + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = start_img.rotate(rotation)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [8]:
def make_contrast_version(path,fname,factor):
        
    new_fname = 'con-'+ str(factor) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + '/' + new_fname
    
    start_img = Image.open(fullpath_start)
    converted = ImageEnhance.Contrast(start_img).enhance(factor)
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [9]:
def make_crop_version(path,fname,factor):
        
    new_fname = 'crop-'+ str(factor) + '_' + fname
    fullpath_start  = path + '/' + fname
    fullpath_finish = path + '/' + new_fname
    
    start_img = Image.open(fullpath_start)
    wid, ht = start_img.size
    (left, upper, right, lower) = (0, 0, wid, ht*factor)
    
    converted = start_img.crop((left, upper, right, lower))
    converted.save(fullpath_finish)
    start_img.close()
    
    return new_fname

In [10]:
def transform_all_in_dir(image_dir):
    count_images = 0
    
    # get files in dir into a list; remove '.DS_Store'
    image_list = rm_DS(os.listdir(image_dir))
    
    print(f"...starting transform of {len(image_list)} images in {image_dir}")
    
    # run transforms on the files in the list
    for image in image_list: 
        count_images += 1

        # Include BW versions to avoid training on color variation
        make_bw_version(image_dir, image)

        # Simple horizontal flip so that we don't inadvertently
        #    train on left-facing are in shallow, right-facing are in deep, or similar
        make_hflip_version(image_dir, image)

        # Rotation of up to 5 degrees may account for handheld horizon variation in photos
        cx = round(np.random.uniform(1,5),1)     
        make_rot_version(image_dir, image, cx)

        # Crops that retained less than 0.8 of the image were too extreme
        cx = round(np.random.uniform(.9,.95),2)
        make_crop_version(image_dir, image, cx)
        
    return count_images

## Make Transforms

In [11]:
# Make list of directories
all_photos_dirs = []

for topd in ['train','validate','test']:
    for caty in ['depth_0','depth_1','depth_2','depth_3','depth_4']:
        ndir = base_dir + '/' + topd + '/' + caty
        all_photos_dirs.append(ndir)
all_photos_dirs

['../images/trucks/train/depth_0',
 '../images/trucks/train/depth_1',
 '../images/trucks/train/depth_2',
 '../images/trucks/train/depth_3',
 '../images/trucks/train/depth_4',
 '../images/trucks/validate/depth_0',
 '../images/trucks/validate/depth_1',
 '../images/trucks/validate/depth_2',
 '../images/trucks/validate/depth_3',
 '../images/trucks/validate/depth_4',
 '../images/trucks/test/depth_0',
 '../images/trucks/test/depth_1',
 '../images/trucks/test/depth_2',
 '../images/trucks/test/depth_3',
 '../images/trucks/test/depth_4']

In [12]:
# Call the augmentation routine for the images in the list
for tdir in all_photos_dirs:
    n = transform_all_in_dir(tdir)
    print(f"{n} photos augmented in {tdir}\n")

...starting transform of 8 images in ../images/trucks/train/depth_0
8 photos augmented in ../images/trucks/train/depth_0

...starting transform of 14 images in ../images/trucks/train/depth_1
14 photos augmented in ../images/trucks/train/depth_1

...starting transform of 13 images in ../images/trucks/train/depth_2
13 photos augmented in ../images/trucks/train/depth_2

...starting transform of 16 images in ../images/trucks/train/depth_3
16 photos augmented in ../images/trucks/train/depth_3

...starting transform of 12 images in ../images/trucks/train/depth_4
12 photos augmented in ../images/trucks/train/depth_4

...starting transform of 6 images in ../images/trucks/validate/depth_0
6 photos augmented in ../images/trucks/validate/depth_0

...starting transform of 7 images in ../images/trucks/validate/depth_1
7 photos augmented in ../images/trucks/validate/depth_1

...starting transform of 10 images in ../images/trucks/validate/depth_2
10 photos augmented in ../images/trucks/validate/depth

In [13]:
print("Additional image creation is complete.")

Additional image creation is complete.
